In [1]:
# Download the dataset
# You may choose where to download the data.

# Google Drive
#!gdown --id '1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy' --output food-11.zip

# Dropbox
# !wget https://www.dropbox.com/s/m9q6273jl3djall/food-11.zip -O food-11.zip

# MEGA
# !sudo apt install megatools
# !megadl "https://mega.nz/#!zt1TTIhK!ZuMbg5ZjGWzWX1I6nEUbfjMZgCmAgeqJlwDkqdIryfg"

# Unzip the dataset.
# This may take some time.
#!unzip -q food-11.zip

In [1]:
#import pytorch
import torch

# torch.backends.cudnn: set CNN algorithmtorch.backends.cudnn
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# get the current available device ('cpu' or 'cuda')
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'
device = get_device()
print(torch.cuda.is_available())
#set random variable
import numpy as np
myseed = 1
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

True


In [3]:
# import torch
# import torch.nn as nn
# m = nn.Conv2d(1, 1, 2 , stride=2, padding=(9,9),bias = False)
# test = torch.tensor([[[1.0,2.0,3.0],[4.0,5.0,6.0],[7.0,8.0,9.0]]])
# print(test)
# print(m(test))

In [9]:
# from PIL import Image
# with Image.open("C:/Users/user/Pictures\Screenshots/資格證.jpg") as im:
#     im.show()

In [3]:
# import torch
# x = torch.FloatTensor([[2,7,1],[3,4,9],[5,6,8]])
# y = x[x> 2]
# print(x,y)
# print(x.argmax(dim = 0))
# print(x.argmax(dim = 1))
# print(x.argmax(dim = -1))

In [2]:
import torchvision.transforms as transforms
from torchvision.datasets import DatasetFolder
from PIL import Image
def get_tfm(mode):
    if mode in ["val","test"]:
      tfm = transforms.Compose([
      transforms.Resize((128, 128)),
      transforms.ToTensor(),
      ])
    elif mode in ["train_unlabeled","train"]:
       tfm = transforms.Compose([
      transforms.Resize((128, 128)),
      transforms.ToTensor(),
      ])
    return tfm
def food_f(mode):
    dataset = DatasetFolder(data_info[mode]["path"], loader=lambda x: Image.open(x), extensions="jpg", transform=get_tfm(mode))
    print('Size of {} data: {}'.format(mode,len(list(dataset))))
    return dataset

#create a dict of functions and path w.r.t. different mode
data_info = {
    "train_origin":{"path":"./food-11/training/labeled"},
    "train":{"path":"./food-11/training/unlabeled"},
    "val":{"path":"./food-11/validation"},
    "test":{"path":"./food-11/testing"}
}

In [32]:

tfm = transforms.Compose([
      transforms.Resize((128, 128)),
      transforms.ToTensor(),
      ])
print(type(tfm))

<class 'torchvision.transforms.transforms.Compose'>


In [36]:
x = 500# x = int(max(prep))
print(list(food_f("val"))[x][1])

Size of val data: 660
8


In [16]:
h_paras = {
    # maximum number of epochs
    'n_epochs': 80,
    # mini-batch size for dataloader
    'batch_size': 128,
    # optimization algorithm (optimizer in torch.optim)
    'optimizer': 'Adam',
    # hyper-parameters for the optimizer (depends on which optimizer you are using)
    'optim_hparas': {
        # learning rate of Adam
        'lr': 0.0003,
        "weight_decay" : 1e-5
    },
    # your model will be saved here
    'save_path': './model.pth'
}

In [4]:
from torch.utils.data import DataLoader

# Construct data loaders.
train_set = DataLoader(food_f("train"), batch_size=h_paras["batch_size"], shuffle=True, num_workers=0, pin_memory=True)
valid_set = DataLoader(food_f("val"), batch_size=h_paras["batch_size"], shuffle=True, num_workers=0, pin_memory=True)
test_set = DataLoader(food_f("test"), batch_size=h_paras["batch_size"], shuffle=False)

Size of train data: 6786
Size of val data: 660
Size of test data: 3347


In [14]:
for i , t in enumerate(valid_set):
    if i ==0:
        test_t = t[0]
#        print(type(t[0]),t[0].size())
        print(test_t.size())
        x = test_t[list(range(64))]
        print(x.size())

torch.Size([128, 3, 128, 128])
torch.Size([64, 3, 128, 128])


In [ ]:
from torch import nn
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # The arguments for commonly used modules:
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)

        # input image size: [3, 128, 128]
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(4, 4, 0),
        )
        self.flat_layer = nn.Flatten()
        self.fc_layers = nn.Sequential(
            nn.Linear(256 * 8 * 8, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 11)
        )

    def forward(self, x):
        # input (x): [batch_size, 3, 128, 128]
        # output: [batch_size, 11]

        # Extract features by convolutional layers.
        x = self.cnn_layers(x)

        # The extracted feature map must be flatten before going to fully-connected layers.
        x = self.flat_layer(x)

        # The features are transformed by fully-connected layers to obtain the final logits.
        x = self.fc_layers(x)
        return x

In [15]:
from tqdm.auto import tqdm
from torch.utils.data import ConcatDataset
def get_pseudo_labels(model, threshold=0.65):
    # This functions generates pseudo-labels of a dataset using given model.
    # It returns an instance of DatasetFolder containing images whose prediction confidences exceed a given threshold.
    # You are NOT allowed to use any models trained on external data for pseudo-labeling.

    # Construct a data loader.
    train_unlabeled_set = \
    DataLoader(food_f("train_unlabeled"), batch_size=h_paras["batch_size"], shuffle=True, num_workers=8, pin_memory=True)
    # Make sure the model is in eval mode.
    model.eval()
    # Define softmax function.
    softmax = nn.Softmax(dim=-1)

    # Iterate over the dataset by batches.
    datas = []
    labels = []
    for batch in tqdm(train_unlabeled_set):
        img = batch[0]
        # Forward the data
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            img_d = img.to(device)
            logits = model(img_d)

        # Obtain the probability distributions by applying softmax on logits.
        predicts = softmax(logits)
        for  row_index , probs in enumerate(predicts):
            max_prob , max_column_index = probs.max(dim = -1)
            if max_prob > threshold:
                datas.append(img[row_index].tolist())
                labels += (max_column_index.tolist())
    new_train_set = torch.FloatTensor(datas)
    new_train_label = torch.LongTensor(labels)
    model.train()
    return new_train_set,new_train_label

d:\Anaconda\envs\ML2021_python_3_11_5\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Initialize a model, and put it on the device specified.
model = Classifier().to(device)
model.device = device
optimizer = torch.optim.Adam(model.parameters(), **h_paras["optim_hparas"])

# The number of training epochs.
n_epochs = 80

# Whether to do semi-supervised learning.
do_semi = False

for epoch in range(n_epochs):
    if do_semi:
        # Obtain pseudo-labels for unlabeled data using trained model.
        pseudo_set = get_pseudo_labels(model)
        concat_dataset = ConcatDataset([train_set, pseudo_set])
        train_set = pseudo_set
        train_loader = DataLoader(concat_dataset, batch_size=128, shuffle=True, num_workers=8, pin_memory=True)

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []

    # Iterate the training set by batches.
    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch

        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs.to(device))

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = h_paras[](logits, labels.to(device))

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)

    # The average loss and accuracy of the training set is the average of the recorded values.
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
          logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

In [ ]:
# Make sure the model is in eval mode.
# Some modules like Dropout or BatchNorm affect if the model is in training mode.
model.eval()

# Initialize a list to store the predictions.
predictions = []

# Iterate the testing set by batches.
for batch in tqdm(test_loader):
    # A batch consists of image data and corresponding labels.
    # But here the variable "labels" is useless since we do not have the ground-truth.
    # If printing out the labels, you will find that it is always 0.
    # This is because the wrapper (DatasetFolder) returns images and labels for each batch,
    # so we have to create fake labels to make it work normally.
    imgs, labels = batch

    # We don't need gradient in testing, and we don't even have labels to compute loss.
    # Using torch.no_grad() accelerates the forward process.
    with torch.no_grad():
        logits = model(imgs.to(device))

    # Take the class with greatest logit as prediction and record it.
    predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

In [ ]:
# Save predictions into the file.
with open("predict.csv", "w") as f:

    # The first row must be "Id, Category"
    f.write("Id,Category\n")

    # For the rest of the rows, each image id corresponds to a predicted class.
    for i, pred in  enumerate(predictions):
         f.write(f"{i},{pred}\n")